In [1]:
###########################################
PROJECT_NAME = 'square_bar'
###########################################

In [2]:
import config #place your Openai key here
from openai import OpenAI

# Initialize the OpenAI client with API key
client = OpenAI(api_key=config.OPENAI_API_KEY)

In [3]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None
model_call_count = 0

# Read the system message once and append to history
with open("system_geo.txt", 'r') as file:
    system_message = file.read().strip()
conversation_history.append({"role": "system", "content": system_message})

In [4]:
#FUNCTIONS FOR INTERACTING WITH CHATBOT
    
def chat_with_bot(user_messages):
    global last_response
    # Add the new user input to the conversation history
    conversation_history.append({"role": "user", "content": user_messages})
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=conversation_history,
        temperature=0
    )
    # Extract the model's response
    model_response = response.choices[0].message.content
    # Update the last response
    last_response = model_response
    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    # Print the model's response
    return print(model_response)

#Saves .geo file from response
def extract_and_save_geo_file(response_text): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [5]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [6]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."), 
# when you are satisfied save the .geo file with
# extract_and_save_geo_file(last_response) and
# visualize it with gmsh software.
# Create mesh with generate_mesh() and
# visualize it with gmsh, Paraview, or
# plot_grid(msh_file) for quick visualization
###############################################

In [7]:
chat_with_bot("""Create a bar with a square section. The axis of the bar is parallel to the z axis
""")

To create a bar with a square section whose axis is parallel to the z-axis, I need to know the length of the bar and the dimensions of the square section. Could you please provide the length of the bar and the side length of the square section?


In [8]:
chat_with_bot("The side length of the square cross-section is 1cm)

Great, now I just need the length of the bar along the z-axis to complete the geometry. Could you please provide that?


In [9]:
chat_with_bot("What is the height of the Mole Antonelliana in Turin, Italy?              ")

The height of the Mole Antonelliana in Turin, Italy, is approximately 167.5 meters. However, this information is not relevant to the creation of the .geo file for the bar with a square section. To proceed with the .geo file, I need the length of the bar along the z-axis. Could you please specify this length?


In [10]:
chat_with_bot("The length of the bar along the z-axis is 10 cm")

Thank you for providing the dimensions. Now I will create the .geo file for a bar with a square cross-section of 1 cm and a length of 10 cm along the z-axis.

//BEGIN_GEO
// Define the dimensions
L = 10; // Length of the bar in cm
a = 1; // Side length of the square cross-section in cm

// Define the points
Point(1) = {0, 0, 0, a};
Point(2) = {a, 0, 0, a};
Point(3) = {a, a, 0, a};
Point(4) = {0, a, 0, a};
Point(5) = {0, 0, L, a};
Point(6) = {a, 0, L, a};
Point(7) = {a, a, L, a};
Point(8) = {0, a, L, a};

// Define the lines
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};
Line(5) = {5, 6};
Line(6) = {6, 7};
Line(7) = {7, 8};
Line(8) = {8, 5};
Line(9) = {1, 5};
Line(10) = {2, 6};
Line(11) = {3, 7};
Line(12) = {4, 8};

// Define the surfaces
Line Loop(1) = {1, 2, 3, 4};
Plane Surface(1) = {1};
Line Loop(2) = {5, 6, 7, 8};
Plane Surface(2) = {2};
Line Loop(3) = {1, 10, -5, -9};
Plane Surface(3) = {3};
Line Loop(4) = {2, 11, -6, -10};
Plane Surface(4) = {4};
Line Loo

In [11]:
extract_and_save_geo_file(last_response)

square_bar.geo file saved
Visualize .geo files with GMSH


In [19]:
generate_mesh(Mesh_Min_Size = 0.05, Mesh_Max_Size = 0.1)

Creating MESH...
 
square_bar.msh saved
You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()
